In [1]:
%load_ext autoreload
%autoreload 2

import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import concurrent.futures

import warnings
from typing import Self
from datetime import datetime
from functools import reduce
from sklearn.metrics import precision_recall_curve, auc, roc_curve
from concurrent.futures import ThreadPoolExecutor, wait, FIRST_COMPLETED

import src.utils.anomalydetectors as m
import src.utils.globals as g
import src.utils.aggregators as agg

from src.utils.experiment import Experiment
from src.utils.noise import NoiseFactory
from src.utils.plotting import plot_rpcurves

In [2]:
def name_experiment(prefix, ratio, intensity, robotids):
    name = f"{prefix}_{ratio}_{intensity}"

    if robotids:
        name = name + f"_{'_'.join(item.strip() for item in robotids)}"

    return name

def name_guassian_experiment(ratio, intensity, robotids):
    return name_experiment("gaussian", ratio, intensity, robotids)

def name_sinusoidal_experiment(ratio, intensity, robotids):
    return name_experiment("sinusoidal", ratio, intensity, robotids)

def name_point_experiment(ratio, intensity, robotids):
    return name_experiment("point", ratio, intensity, robotids)

In [3]:
df_in = pd.read_parquet(g.path_tracks)

In [4]:
# Boolean whether to run the experiments or load
load_experiments = False 

# Running Experiment Parameters
models = [m.ZScore(), m.MZScore(), m.LOF(), m.IF()]
exp_columns = ['torqueactual']
syn_column = 'torqueactual'

guassian_params     = [0.10, 0.25, 0.40, 0.55]
guassian_column_length = len(guassian_params)

sinusoidal_params   = [0.25, 0.5, 1, 2]
sinusoidal_column_length = len(sinusoidal_params)

point_params        = [1, 5, 10, 100]
point_column_length = len(point_params)

ids = None #ids = ['6640-102140-1']
synratio = 0.1

# Loading Experiment Parameters


# Loading Experiments

In [ ]:
def load_experiments(params, name_func, exp_dict):
    for intensity in params:
        name = name_func(synratio, intensity, ids)
        print(f"Loading Experiment {name}")
        exp_dict[name] = Experiment.unpickle(name)
    print("Experiments Loaded")

# Load or Run Experiments
exps = {}
load_experiments(guassian_params, name_guassian_experiment, exps)
load_experiments(sinusoidal_params, name_sinusoidal_experiment, exps)
load_experiments(point_params, name_point_experiment, exps)

In [ ]:
def plot_multiple(plots, title="", colnums=3):
    # Calculate the number of rows needed
    cols = colnums
    num_plots = len(plots)
    rows = (num_plots + cols - 1) // cols  # Ceiling division to ensure enough rows

    # Create a figure with subplots
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 5*rows))

    # Flatten axes for easy indexing if needed
    axes = axes.flatten()

    # Loop through the DataFrames and plot each in the corresponding subplot
    for i, fn in enumerate(plots):
        fn(axes[i])

    # Hide any unused subplots
    for j in range(i+1, len(axes)):
        fig.delaxes(axes[j])

    plt.suptitle(title)
    plt.tight_layout()

    # Show the plot
    plt.show()

def _plot_experiments(exp, model, ax, id=None):
    df = exp.results['df'].sort_values(by='timeindex')
    df = df[df['seqid'] == id]

    ax.plot(df['timeindex'], df['torqueactual'], label=model)
    ax.plot(df['timeindex'], df[model], label='Anomaly Score')

    ax.xlabel('Time (s))')
    ax.ylabel('Torque (nm)')
    ax.title(f'ID : {id} - Model: {model}')
    ax.legend()
    ax.grid(True)

exp = exps['gaussian_0.1_0.25']
print(exp.results['df'])
plot_multiple([lambda ax: _plot_experiments(exp, 'z', ax, np.random.choice(exp.results['df']['seqid'].unique()))], title="ZScore")

NameError: name 'exps' is not defined